# 1 Layer Creation: Here we create the neural network with one hidden layer.

# We create the neural network as the index assignment function from scratch.

In [51]:
# 79-character line limit
######### ######### ######### ######### ######### ######### ######### #########
%reset -f

import numpy as np # The NumPy library
#import math # The math module, np includes it
from scipy.integrate import quad # Method for integration in scipy.integrate sub-package


cells_num = 16

# The opitimized spaces for a Guassian distribution in a uniform quantizer(UQ).
# The sizes are for UQs for number of cells: 4, 8, 16, 32, 64, 128, 256.
all_size_gaus = np.array([0.9957, 0.5860, 0.3352, 0.1881, 0.1041, 0.0569, 0.0308])

cell_size = all_size_gaus[int(np.log2(cells_num))-2]

# np.arange has rounding error issue.
# boundaries_symm = np.arange(-(cells_num/2-1) * cell_size, # For symmetric distributions
#                              (cells_num/2) * cell_size,
#                             cell_size) 

boundaries_symm = np.linspace(-(cells_num/2-1) * cell_size, # For symmetric distributions
                               (cells_num/2-1) * cell_size,
                              cells_num-1)
n_inf = float("-inf")
p_inf = float("inf")

boundaries_symm = np.insert(boundaries_symm, 0, n_inf)
boundaries_symm = np.append(boundaries_symm, p_inf)

def pdf(x): # Defining the distribution
    gaus_std = 1
    gaus_mean = 0
    pdf = 1/(gaus_std*np.sqrt(2*np.pi)) * \
                  np.exp(-0.5*((x-gaus_mean)/gaus_std)**2) # Gaussian pdf
    return pdf

def xpdf(x):
    xpdf = x * pdf(x)
    return xpdf

def x2pdf (x):
    x2pdf = x * xpdf(x)
    return x2pdf

prbs = []
xprbs = []
x2prbs = []
cell_reps = []
for i in range (0, cells_num):
    cell_prb, integ_err = quad(pdf, boundaries_symm[i], boundaries_symm[i+1])
    prbs = np.append(prbs, cell_prb)
    
    cell_xprb, integ_err = quad(xpdf, boundaries_symm[i], boundaries_symm[i+1])
    xprbs = np.append(xprbs, cell_xprb)
    
    cell_x2prb, integ_err = quad(x2pdf, boundaries_symm[i], boundaries_symm[i+1])
    x2prbs = np.append(x2prbs, cell_x2prb)
    
    cell_rep = cell_xprb / cell_prb
    cell_reps = np.append(cell_reps, cell_rep)
    
  
  

######### ######### ######### ######### ######### ######### ######### #########
l1_size = cells_num

# # It is the weights and biases initialization.
# l1_weights = (np.random.rand(l1_size, cells_num)-0.5)
# # l1_biases = np.random.rand(l1_size,)-0.5
# l1_biases = np.zeros(l1_size,)
# l2_weights = (np.random.rand(cells_num, l1_size)-0.5)


# It is the "He Initialization" technique.
l1_weights = np.random.randn(l1_size, cells_num) * np.sqrt(2/cells_num)
l1_biases = np.zeros(l1_size,)
l2_weights = np.random.randn(cells_num, l1_size) * np.sqrt(2/l1_size)



# We define the activation functions.
def relu(x):
    return np.maximum(0,x)

def lrelu(x):
    return np.where(x > 0, x, x * 0.1)

def sigmoid(x):
    return 1/(1 + np.exp(-x)) 


l1_outs = relu(np.dot(l1_weights, cell_reps)+l1_biases)
l2_outs = np.dot(l2_weights, l1_outs)
l2_outs = np.round(l2_outs, 16)

# We assigned the indexes based on the final layer outputs.
sorted_outs = np.sort(l2_outs)
indexes = []
for i in range(0, cells_num):
    index = np.where(sorted_outs == l2_outs[i])
    indexes = np.append(indexes, index)

print(indexes)

yijs = [] # Yijs based on the assigned indexes
dijs = []
for i in range(0, int(cells_num/2)):
    j = i + cells_num/2
    celli = np.where(indexes == i)
    cellj = np.where(indexes == j)
    
    yij = (xprbs[celli] + xprbs[cellj])/(prbs[celli] + prbs[cellj])
    yijs = np.append(yijs, yij)
    
    dij = x2prbs[celli] + yij**2 * prbs[celli] - yij *2 * xprbs[celli]\
        + x2prbs[cellj] + yij**2 * prbs[cellj] - yij *2 * xprbs[cellj]
    dijs = np.append(dijs, dij)
    
distortion = np.round(sum(dijs), 16) # The Eve's distortion based on the assigned indexes

print(distortion)

past_velocity1 = 0 # Past velocity for momentum
past_velocity_b1 = 0
past_velocity2 = 0
past_velocity_b2 = 0
past_velocity3 = 0


m_adam1 = 0 # Adam algorithm parameters
v_adam1 = 0
m_adam_b1 = 0
v_adam_b1 = 0

m_adam2 = 0
v_adam2 = 0
m_adam_b2 = 0
v_adam_b2 = 0

m_adam3 = 0
v_adam3 = 0

[11. 13.  8. 15. 10.  3.  9.  2.  6.  1.  0.  7. 14.  4.  5. 12.]
0.6469959970475995


# 1 Layer Training: Here we train the neural network with one hidden layer.

In [54]:
# This part provides the derivative of the distortion w.r.t. the weights
# for the back propagation. The prefix 'b' shows that the variables are
# calculated temporarily for the gradient descent

learning_rate = 0.5
epsilon = 0.5
 
momentum = 0.1

beta_1 = 0.999 # Parameters of Adam
beta_2 = 0.9

for ii in range (1, 100):
    l1_gradients = [] # Gradients of layer 1 weights
    # learning_rate = 0.1/ii
    for i in range (0, l1_size):
        for j in range (0, cells_num):
            b_l1_weights = l1_weights.copy()
            b_l1_weights[i][j] = l1_weights [i][j] + epsilon
            b_l1_outs = relu(np.dot(b_l1_weights, cell_reps)+l1_biases)
            b_l2_outs = np.dot(l2_weights, b_l1_outs)
            b_l2_outs = np.round(b_l2_outs, 16)
            
            for p in range(0, cells_num-1):
                for q in range (p+1, cells_num):
                    if b_l2_outs[p] == b_l2_outs[q]:
                        b_l2_outs[q] += np.random.rand(1,1)-0.5
            
            b_sorted_outs = np.sort(b_l2_outs)
            b_indexes = []
            for k in range(0, cells_num):
                b_index = np.where(b_sorted_outs == b_l2_outs[k])
                b_indexes = np.append(b_indexes, b_index)

            b_yijs = [] 
            b_dijs = []
            for m in range(0, int(cells_num/2)):
                n = m + cells_num/2
                b_celli = np.where(b_indexes == m)
                b_cellj = np.where(b_indexes == n)

                b_yij = (xprbs[b_celli] + xprbs[b_cellj])/(prbs[b_celli] + prbs[b_cellj])
                b_yijs = np.append(b_yijs, b_yij)

                b_dij = x2prbs[b_celli] + b_yij**2 * prbs[b_celli] - b_yij *2 * xprbs[b_celli]\
                      + x2prbs[b_cellj] + b_yij**2 * prbs[b_cellj] - b_yij *2 * xprbs[b_cellj]
                b_dijs = np.append(b_dijs, b_dij)
                        
            b_distortion = np.round(sum(b_dijs), 16) 
            l1_gradient = (b_distortion - distortion)/epsilon
            l1_gradients = np.append(l1_gradients, l1_gradient)
            
    l1_gradients = np.reshape(l1_gradients, [l1_size, cells_num])
        
        
    l1_biases_gradients = [] # Gradients of the layer 1 biases
    for i in range (0, l1_size):
        b_l1_biases = l1_biases.copy()
        b_l1_biases [i] = l1_biases [i] + epsilon
        b_l1_outs = relu(np.dot(l1_weights, cell_reps)+b_l1_biases)
        b_l2_outs = np.dot(l2_weights, b_l1_outs)
        b_l2_outs = np.round(b_l2_outs, 16)
        
        for p in range(0, cells_num-1):
            for q in range (p+1, cells_num):
                if b_l2_outs[p] == b_l2_outs[q]:
                    b_l2_outs[q] += np.random.rand(1,1)-0.5

        b_sorted_outs = np.sort(b_l2_outs)
        b_indexes = []
        for k in range(0, cells_num):
            b_index = np.where(b_sorted_outs == b_l2_outs[k])
            b_indexes = np.append(b_indexes, b_index)

        b_yijs = [] 
        b_dijs = []
        for m in range(0, int(cells_num/2)):
            n = m + cells_num/2
            b_celli = np.where(b_indexes == m)
            b_cellj = np.where(b_indexes == n)

            b_yij = (xprbs[b_celli] + xprbs[b_cellj])/(prbs[b_celli] + prbs[b_cellj])
            b_yijs = np.append(b_yijs, b_yij)

            b_dij = x2prbs[b_celli] + b_yij**2 * prbs[b_celli] - b_yij *2 * xprbs[b_celli]\
                      + x2prbs[b_cellj] + b_yij**2 * prbs[b_cellj] - b_yij *2 * xprbs[b_cellj]
            b_dijs = np.append(b_dijs, b_dij)
                        
        b_distortion = np.round(sum(b_dijs), 16) 
        l1_biases_gradient = (b_distortion - distortion)/epsilon
        l1_biases_gradients = np.append(l1_biases_gradients, l1_biases_gradient)
    
           
    l2_gradients = [] # Gradients of layer 2 weights
    for i in range (0, cells_num):
        for j in range (0, l1_size):
            b_l2_weights = l2_weights.copy()
            b_l2_weights[i][j] = l2_weights [i][j] + epsilon
            b_l1_outs = relu(np.dot(l1_weights, cell_reps)+l1_biases)
            b_l2_outs = np.dot(b_l2_weights, b_l1_outs)
            b_l2_outs = np.round(b_l2_outs, 16)
            
            for p in range(0, cells_num-1):
                for q in range (p+1, cells_num):
                    if b_l2_outs[p] == b_l2_outs[q]:
                        b_l2_outs[q] += np.random.rand(1,1)-0.5


            b_sorted_outs = np.sort(b_l2_outs)
            b_indexes = []
            for k in range(0, cells_num):
                b_index = np.where(b_sorted_outs == b_l2_outs[k])
                b_indexes = np.append(b_indexes, b_index)

            b_yijs = [] 
            b_dijs = []
            for m in range(0, int(cells_num/2)):
                n = m + cells_num/2
                b_celli = np.where(b_indexes == m)
                b_cellj = np.where(b_indexes == n)

                b_yij = (xprbs[b_celli] + xprbs[b_cellj])/(prbs[b_celli] + prbs[b_cellj])
                b_yijs = np.append(b_yijs, b_yij)

                b_dij = x2prbs[b_celli] + b_yij**2 * prbs[b_celli] - b_yij *2 * xprbs[b_celli]\
                      + x2prbs[b_cellj] + b_yij**2 * prbs[b_cellj] - b_yij *2 * xprbs[b_cellj]
                b_dijs = np.append(b_dijs, b_dij)
                        
            b_distortion = np.round(sum(b_dijs), 16) 
            l2_gradient = (b_distortion - distortion)/epsilon
            l2_gradients = np.append(l2_gradients, l2_gradient)
            
    l2_gradients = np.reshape(l2_gradients, [cells_num, l1_size])

# Now, we update the weights.   
    l1_weights = l1_weights + learning_rate * l1_gradients
    l1_biases = l1_biases + learning_rate * l1_biases_gradients
    l2_weights = l2_weights + learning_rate * l2_gradients
    
#     # We update the weights with momentum.
#     velocity1 = past_velocity1 * momentum + learning_rate *  l1_gradients
#     l1_weights = l1_weights - momentum * velocity1 + learning_rate * l1_gradients
#     past_velocity1 = velocity1 
    
#     velocity_b1 = past_velocity_b1 * momentum + learning_rate *  l1_biases_gradients
#     l1_biases = l1_biases - momentum * velocity_b1 + learning_rate * l1_biases_gradients
#     past_velocity_b1 = velocity_b1
    
#     velocity2 = past_velocity2 * momentum + learning_rate *  l2_gradients
#     l2_weights = l2_weights - momentum * velocity2 + learning_rate * l2_gradients
#     past_velocity2 = velocity2
    
#     # momentum way 2
#     velocity1 = learning_rate * l1_gradients + momentum * past_velocity1
#     l1_weights = l1_weights + velocity1
#     past_velocity1 = velocity1
    
#     velocity2 = learning_rate * l2_gradients + momentum * past_velocity2
#     l2_weights = l2_weights + velocity2
#     past_velocity2 = velocity2
    
#     velocity_b1 = learning_rate * l1_biases_gradients + momentum * past_velocity_b1
#     l1_biases = l1_biases + velocity_b1
#     past_velocity_b1 = velocity_b1

# # Here, we implement the Adam algorithm.

#     m_adam1 = beta_1 * m_adam1 + (1 - beta_1) * l1_gradients
#     v_adam1 = beta_2 * v_adam1 + (1 - beta_2) * np.power(l1_gradients, 2)
#     m_hat1 = m_adam1 / (1 - np.power(beta_1, ii))
#     v_hat1 = v_adam1 / (1 - np.power(beta_2, ii))
#     l1_weights = l1_weights + learning_rate * m_hat1 / (np.sqrt(v_hat1) + epsilon)
    
#     m_adam_b1 = beta_1 * m_adam_b1 + (1 - beta_1) * l1_biases_gradients
#     v_adam_b1 = beta_2 * v_adam_b1 + (1 - beta_2) * np.power(l1_biases_gradients, 2)
#     m_hat_b1 = m_adam_b1 / (1 - np.power(beta_1, ii))
#     v_hat_b1 = v_adam_b1 / (1 - np.power(beta_2, ii))
#     l1_biases = l1_biases + learning_rate * m_hat_b1 / (np.sqrt(v_hat_b1) + epsilon)
    
#     m_adam2 = beta_1 * m_adam2 + (1 - beta_1) * l2_gradients
#     v_adam2 = beta_2 * v_adam2 + (1 - beta_2) * np.power(l2_gradients, 2)
#     m_hat2 = m_adam2 / (1 - np.power(beta_1, ii))
#     v_hat2 = v_adam2 / (1 - np.power(beta_2, ii))
#     l2_weights = l2_weights + learning_rate * m_hat2 / (np.sqrt(v_hat2) + epsilon)
      
    
    l1_outs = relu(np.dot(l1_weights,cell_reps)+l1_biases)
    l2_outs = np.dot(l2_weights, l1_outs)
    l2_outs = np.round(l2_outs, 16)
    
    for p in range(0, cells_num-1):
        for q in range (p+1, cells_num):
            if l2_outs[p] == l2_outs[q]:
                l2_outs[q] += np.random.rand(1,1)-0.5
   

    # We assigned the indexes based on the final layer outputs.
    sorted_outs = np.sort(l2_outs)
    indexes = []
    for i in range(0, cells_num):
        index = np.where(sorted_outs == l2_outs[i])
        indexes = np.append(indexes, index)

    #print(indexes)
    
    yijs = [] # Yijs based on the assigned indexes
    dijs = []
    for i in range(0, int(cells_num/2)):
        j = i + cells_num/2
        celli = np.where(indexes == i)
        cellj = np.where(indexes == j)

        yij = (xprbs[celli] + xprbs[cellj])/(prbs[celli] + prbs[cellj])
        yijs = np.append(yijs, yij)

        dij = x2prbs[celli] + yij**2 * prbs[celli] - yij *2 * xprbs[celli]\
            + x2prbs[cellj] + yij**2 * prbs[cellj] - yij *2 * xprbs[cellj]
        dijs = np.append(dijs, dij)

    distortion = np.round(sum(dijs), 16) # The Eve's distortion based on the assigned indexes
    print(distortion)

0.9875373046650608
0.9875373046650608
0.9875373046650608
0.9875373046650608
0.9875373046650608
0.9875373046650608
0.9875373046650608
0.9875373046650608
0.9875373046650608
0.9865424420062542
0.9865424420062542
0.9865424420062542
0.9865424420062542
0.9865424420062542
0.9865424420062542
0.9865424420062542
0.9875373046650608
0.9875373046650608
0.9875373046650608
0.9875373046650608
0.9875373046650608
0.9875373046650608
0.9875373046650608
0.9875373046650608
0.9875373046650608
0.9875373046650608
0.9875373046650608
0.9875373046650608
0.9875373046650608
0.9875373046650608
0.9875373046650608
0.9875373046650608
0.9875373046650608
0.9875373046650608
0.9865424420062542
0.9865424420062542
0.9865424420062542
0.9875373046650608
0.9875373046650608
0.9875373046650608
0.9875373046650608
0.9875373046650608
0.9875373046650608
0.9875373046650608
0.9875373046650608
0.9875373046650608
0.9875373046650608
0.9875373046650608
0.9875373046650608
0.9875373046650608
0.9865424420062542
0.9865424420062542
0.9865424420

# 2 Layer Creation: Here we create the neural network with two hidden layers.

In [58]:
# 79-character line limit
######### ######### ######### ######### ######### ######### ######### #########
%reset -f

import numpy as np # The NumPy library
#import math # The math module, np includes it
from scipy.integrate import quad # Method for integration in scipy.integrate sub-package


cells_num = 16

# The opitimized spaces for a Guassian distribution in a uniform quantizer(UQ).
# The sizes are for UQs for number of cells: 4, 8, 16, 32, 64, 128, 256.
all_size_gaus = np.array([0.9957, 0.5860, 0.3352, 0.1881, 0.1041, 0.0569, 0.0308])

cell_size = all_size_gaus[int(np.log2(cells_num))-2]

# np.arange has rounding error issue.
# boundaries_symm = np.arange(-(cells_num/2-1) * cell_size, # For symmetric distributions
#                              (cells_num/2) * cell_size,
#                             cell_size) 

boundaries_symm = np.linspace(-(cells_num/2-1) * cell_size, # For symmetric distributions
                               (cells_num/2-1) * cell_size,
                              cells_num-1)
n_inf = float("-inf")
p_inf = float("inf")

boundaries_symm = np.insert(boundaries_symm, 0, n_inf)
boundaries_symm = np.append(boundaries_symm, p_inf)

def pdf(x): # Defining the distribution
    gaus_std = 1
    gaus_mean = 0
    pdf = 1/(gaus_std*np.sqrt(2*np.pi)) * \
                  np.exp(-0.5*((x-gaus_mean)/gaus_std)**2) # Gaussian pdf
    return pdf

def xpdf(x):
    xpdf = x * pdf(x)
    return xpdf

def x2pdf (x):
    x2pdf = x * xpdf(x)
    return x2pdf

prbs = []
xprbs = []
x2prbs = []
cell_reps = []
for i in range (0, cells_num):
    cell_prb, integ_err = quad(pdf, boundaries_symm[i], boundaries_symm[i+1])
    prbs = np.append(prbs, cell_prb)
    
    cell_xprb, integ_err = quad(xpdf, boundaries_symm[i], boundaries_symm[i+1])
    xprbs = np.append(xprbs, cell_xprb)
    
    cell_x2prb, integ_err = quad(x2pdf, boundaries_symm[i], boundaries_symm[i+1])
    x2prbs = np.append(x2prbs, cell_x2prb)
    
    cell_rep = cell_xprb / cell_prb
    cell_reps = np.append(cell_reps, cell_rep)
    

######### ######### ######### ######### ######### ######### ######### #########
l1_size = cells_num 
l2_size = cells_num

# # It is the weights and biases initialization.
# l1_weights = np.random.rand(l1_size, cells_num)-0.5
# l1_biases = np.random.rand(l1_size,)-0.5
# l2_weights = np.random.rand(l2_size, l1_size)-0.5
# l2_biases = np.random.rand(l2_size,)-0.5
# l3_weights = np.random.rand(cells_num, l2_size)-0.5

# It is the "He Initialization" technique.
l1_weights = np.random.randn(l1_size, cells_num) * np.sqrt(2/cells_num)
l1_biases = np.zeros(l1_size,)
l2_weights = np.random.randn(l2_size, l1_size) * np.sqrt(2/l1_size)
l2_biases = np.zeros(l2_size,)
l3_weights = np.random.randn(cells_num, l2_size) * np.sqrt(2/l2_size)

# We define the activation functions.
def relu(x):
    return np.maximum(0,x)

l1_outs = relu(np.dot(l1_weights, cell_reps)+l1_biases)
l2_outs = relu(np.dot(l2_weights, l1_outs)+l2_biases)
l3_outs = np.dot(l3_weights, l2_outs)
l3_outs = np.round(l3_outs, 16)

# We assigned the indexes based on the final layer outputs.
sorted_outs = np.sort(l3_outs)
indexes = []
for i in range(0, cells_num):
    index = np.where(sorted_outs == l3_outs[i])
    indexes = np.append(indexes, index)

print(indexes)

yijs = [] # Yijs based on the assigned indexes
dijs = []
for i in range(0, int(cells_num/2)):
    j = i + cells_num/2
    celli = np.where(indexes == i)
    cellj = np.where(indexes == j)
    
    yij = (xprbs[celli] + xprbs[cellj])/(prbs[celli] + prbs[cellj])
    yijs = np.append(yijs, yij)
    
    dij = x2prbs[celli] + yij**2 * prbs[celli] - yij *2 * xprbs[celli]\
        + x2prbs[cellj] + yij**2 * prbs[cellj] - yij *2 * xprbs[cellj]
    dijs = np.append(dijs, dij)
    
distortion = np.round(sum(dijs), 16) # The Eve's distortion based on the assigned indexes

print(distortion)

past_velocity1 = 0 # Past velocity for momentum
past_velocity_b1 = 0
past_velocity2 = 0
past_velocity_b2 = 0
past_velocity3 = 0


m_adam1 = 0
v_adam1 = 0
m_adam_b1 = 0
v_adam_b1 = 0

m_adam2 = 0
v_adam2 = 0
m_adam_b2 = 0
v_adam_b2 = 0

m_adam3 = 0
v_adam3 = 0

[11.  9.  8. 12. 13.  0. 14.  7. 10.  6.  4.  5. 15.  2.  3.  1.]
0.8313416094946148


# 2 Layers Training: Here we train the neural network with two hidden layers.

In [60]:
# This part provides the derivative of the distortion w.r.t. the weights
# for the back propagation. The prefix 'b' shows that the variables are
# calculated temporarily for gradient descent

momentum = 0.9 # Momentum for momentum algorithm

beta_1 = 0.9 # Parameters of Adam
beta_2 = 0.999


learning_rate = 0.5
epsilon = 0.5

iteration_num = 100
for ii in range (1, iteration_num):
    l1_gradients = [] # Gradients of layer 1 weights
    for i in range (0, l1_size):
        for j in range (0, cells_num):
            b_l1_weights = l1_weights.copy()
            b_l1_weights[i][j] = l1_weights [i][j] + epsilon
            b_l1_outs = relu(np.dot(b_l1_weights, cell_reps)+l1_biases)
            b_l2_outs = relu(np.dot(l2_weights, b_l1_outs)+l2_biases)
            b_l3_outs = np.dot(l3_weights, b_l2_outs)
            b_l3_outs = np.round(b_l3_outs, 16)
            
            for p in range(0, cells_num-1):
                for q in range (p+1, cells_num):
                    if b_l3_outs[p] == b_l3_outs[q]:
                        b_l3_outs[q] += np.random.rand(1,1)-0.5
            
            b_sorted_outs = np.sort(b_l3_outs)
            b_indexes = []
            for k in range(0, cells_num):
                b_index = np.where(b_sorted_outs == b_l3_outs[k])
                b_indexes = np.append(b_indexes, b_index)

            b_yijs = [] 
            b_dijs = []
            for m in range(0, int(cells_num/2)):
                n = m + cells_num/2
                b_celli = np.where(b_indexes == m)
                b_cellj = np.where(b_indexes == n)

                b_yij = (xprbs[b_celli] + xprbs[b_cellj])/(prbs[b_celli] + prbs[b_cellj])
                b_yijs = np.append(b_yijs, b_yij)

                b_dij = x2prbs[b_celli] + b_yij**2 * prbs[b_celli] - b_yij *2 * xprbs[b_celli]\
                      + x2prbs[b_cellj] + b_yij**2 * prbs[b_cellj] - b_yij *2 * xprbs[b_cellj]
                b_dijs = np.append(b_dijs, b_dij)
                        
            b_distortion = np.round(sum(b_dijs), 16) 
            l1_gradient = (b_distortion - distortion)/epsilon
            l1_gradients = np.append(l1_gradients, l1_gradient)
            
    l1_gradients = np.reshape(l1_gradients, [cells_num, cells_num])
        
        
    l1_biases_gradients = [] # Gradients of the layer 1 biases
    for i in range (0, l1_size):
        b_l1_biases = l1_biases.copy()
        b_l1_biases [i] = l1_biases [i] + epsilon
        b_l1_outs = relu(np.dot(l1_weights, cell_reps)+b_l1_biases)
        b_l2_outs = relu(np.dot(l2_weights, b_l1_outs)+l2_biases)
        b_l3_outs = np.dot(l3_weights, b_l2_outs)
        b_l3_outs = np.round(b_l3_outs, 16)
        
        for p in range(0, cells_num-1):
            for q in range (p+1, cells_num):
                if b_l3_outs[p] == b_l3_outs[q]:
                    b_l3_outs[q] += np.random.rand(1,1)-0.5

        b_sorted_outs = np.sort(b_l3_outs)
        b_indexes = []
        for k in range(0, cells_num):
            b_index = np.where(b_sorted_outs == b_l3_outs[k])
            b_indexes = np.append(b_indexes, b_index)

        b_yijs = [] 
        b_dijs = []
        for m in range(0, int(cells_num/2)):
            n = m + cells_num/2
            b_celli = np.where(b_indexes == m)
            b_cellj = np.where(b_indexes == n)

            b_yij = (xprbs[b_celli] + xprbs[b_cellj])/(prbs[b_celli] + prbs[b_cellj])
            b_yijs = np.append(b_yijs, b_yij)

            b_dij = x2prbs[b_celli] + b_yij**2 * prbs[b_celli] - b_yij *2 * xprbs[b_celli]\
                      + x2prbs[b_cellj] + b_yij**2 * prbs[b_cellj] - b_yij *2 * xprbs[b_cellj]
            b_dijs = np.append(b_dijs, b_dij)
                        
        b_distortion = np.round(sum(b_dijs), 16) 
        l1_biases_gradient = (b_distortion - distortion)/epsilon
        l1_biases_gradients = np.append(l1_biases_gradients, l1_biases_gradient)
    
           
    l2_gradients = [] # Gradients of layer 2 weights
    for i in range (0, l2_size):
        for j in range (0, l1_size):
            b_l2_weights = l2_weights.copy()
            b_l2_weights[i][j] = l2_weights [i][j] + epsilon
            b_l1_outs = relu(np.dot(l1_weights, cell_reps)+l1_biases)
            b_l2_outs = relu(np.dot(b_l2_weights, b_l1_outs)+l2_biases)
            b_l3_outs = np.dot(l3_weights, b_l2_outs)
            b_l3_outs = np.round(b_l3_outs, 16)
            
            for p in range(0, cells_num):
                for q in range (p+1, cells_num):
                    if b_l3_outs[p] == b_l3_outs[q]:
                        b_l3_outs[q] += np.random.rand(1,1)-0.5


            b_sorted_outs = np.sort(b_l3_outs)
            b_indexes = []
            for k in range(0, cells_num):
                b_index = np.where(b_sorted_outs == b_l3_outs[k])
                b_indexes = np.append(b_indexes, b_index)

            b_yijs = [] 
            b_dijs = []
            for m in range(0, int(cells_num/2)):
                n = m + cells_num/2
                b_celli = np.where(b_indexes == m)
                b_cellj = np.where(b_indexes == n)

                b_yij = (xprbs[b_celli] + xprbs[b_cellj])/(prbs[b_celli] + prbs[b_cellj])
                b_yijs = np.append(b_yijs, b_yij)

                b_dij = x2prbs[b_celli] + b_yij**2 * prbs[b_celli] - b_yij *2 * xprbs[b_celli]\
                      + x2prbs[b_cellj] + b_yij**2 * prbs[b_cellj] - b_yij *2 * xprbs[b_cellj]
                b_dijs = np.append(b_dijs, b_dij)
                        
            b_distortion = np.round(sum(b_dijs), 16) 
            l2_gradient = (b_distortion - distortion)/epsilon
            l2_gradients = np.append(l2_gradients, l2_gradient)
            
    l2_gradients = np.reshape(l2_gradients, [cells_num, cells_num])
    
    
    l2_biases_gradients = [] # Gradients of the layer 2 biases
    for i in range (0, l2_size):
        b_l2_biases = l2_biases.copy()
        b_l2_biases [i] = l2_biases [i] + epsilon
        b_l1_outs = relu(np.dot(l1_weights, cell_reps)+l1_biases)
        b_l2_outs = relu(np.dot(l2_weights, b_l1_outs)+b_l2_biases)
        b_l3_outs = np.dot(l3_weights, b_l2_outs)
        b_l3_outs = np.round(b_l3_outs, 16)
        
        for p in range(0, cells_num-1):
            for q in range (p+1, cells_num):
                if b_l3_outs[p] == b_l3_outs[q]:
                    b_l3_outs[q] += np.random.rand(1,1)-0.5

        b_sorted_outs = np.sort(b_l3_outs)
        b_indexes = []
        for k in range(0, cells_num):
            b_index = np.where(b_sorted_outs == b_l3_outs[k])
            b_indexes = np.append(b_indexes, b_index)

        b_yijs = [] 
        b_dijs = []
        for m in range(0, int(cells_num/2)):
            n = m + cells_num/2
            b_celli = np.where(b_indexes == m)
            b_cellj = np.where(b_indexes == n)

            b_yij = (xprbs[b_celli] + xprbs[b_cellj])/(prbs[b_celli] + prbs[b_cellj])
            b_yijs = np.append(b_yijs, b_yij)

            b_dij = x2prbs[b_celli] + b_yij**2 * prbs[b_celli] - b_yij *2 * xprbs[b_celli]\
                      + x2prbs[b_cellj] + b_yij**2 * prbs[b_cellj] - b_yij *2 * xprbs[b_cellj]
            b_dijs = np.append(b_dijs, b_dij)
                        
        b_distortion = np.round(sum(b_dijs), 16) 
        l2_biases_gradient = (b_distortion - distortion)/epsilon
        l2_biases_gradients = np.append(l2_biases_gradients, l2_biases_gradient)
        
        
    l3_gradients = [] # Gradients of layer 3 weights
    for i in range (0, cells_num):
        for j in range (0, l2_size):
            b_l3_weights = l3_weights.copy()
            b_l3_weights[i][j] = l3_weights [i][j] + epsilon
            b_l1_outs = relu(np.dot(l1_weights, cell_reps)+l1_biases)
            b_l2_outs = relu(np.dot(l2_weights, b_l1_outs)+l2_biases)
            b_l3_outs = np.dot(b_l3_weights, b_l2_outs)
            b_l3_outs = np.round(b_l3_outs, 16)
            
            for p in range(0, cells_num):
                for q in range (p+1, cells_num):
                    if b_l3_outs[p] == b_l3_outs[q]:
                        b_l3_outs[q] += np.random.rand(1,1)-0.5


            b_sorted_outs = np.sort(b_l3_outs)
            b_indexes = []
            for k in range(0, cells_num):
                b_index = np.where(b_sorted_outs == b_l3_outs[k])
                b_indexes = np.append(b_indexes, b_index)

            b_yijs = [] 
            b_dijs = []
            for m in range(0, int(cells_num/2)):
                n = m + cells_num/2
                b_celli = np.where(b_indexes == m)
                b_cellj = np.where(b_indexes == n)

                b_yij = (xprbs[b_celli] + xprbs[b_cellj])/(prbs[b_celli] + prbs[b_cellj])
                b_yijs = np.append(b_yijs, b_yij)

                b_dij = x2prbs[b_celli] + b_yij**2 * prbs[b_celli] - b_yij *2 * xprbs[b_celli]\
                      + x2prbs[b_cellj] + b_yij**2 * prbs[b_cellj] - b_yij *2 * xprbs[b_cellj]
                b_dijs = np.append(b_dijs, b_dij)
                        
            b_distortion = np.round(sum(b_dijs), 16) 
            l3_gradient = (b_distortion - distortion)/epsilon
            l3_gradients = np.append(l3_gradients, l3_gradient)
            
    l3_gradients = np.reshape(l3_gradients, [cells_num, cells_num])
        
#     # Now, we update the weights.   
#     l1_weights = l1_weights + learning_rate * l1_gradients
#     l1_biases = l1_biases + learning_rate * l1_biases_gradients
#     l2_weights = l2_weights + learning_rate * l2_gradients    
#     l2_biases = l2_biases + learning_rate * l2_biases_gradients
#     l3_weights = l3_weights + learning_rate * l3_gradients
    
#     # We update the weights with momentum.
#     velocity1 = past_velocity1 * momentum + learning_rate *  l1_gradients
#     l1_weights = l1_weights - momentum * velocity1 + learning_rate * l1_gradients
#     past_velocity1 = velocity1 
    
#     velocity_b1 = past_velocity_b1 * momentum + learning_rate *  l1_biases_gradients
#     l1_biases = l1_biases - momentum * velocity_b1 + learning_rate * l1_biases_gradients
#     past_velocity_b1 = velocity_b1
    
#     velocity2 = past_velocity2 * momentum + learning_rate *  l2_gradients
#     l2_weights = l2_weights - momentum * velocity2 + learning_rate * l2_gradients
#     past_velocity2 = velocity2
    
#     velocity_b2 = past_velocity_b2 * momentum + learning_rate *  l2_biases_gradients
#     l2_biases = l2_biases - momentum * velocity_b2 + learning_rate * l2_biases_gradients
#     past_velocity_b2 = velocity_b2
    
#     velocity3 = past_velocity3 * momentum + learning_rate *  l3_gradients
#     l3_weights = l3_weights - momentum * velocity3 + learning_rate * l3_gradients
#     past_velocity3 = velocity3



    # momentum way 2
    velocity1 = learning_rate * l1_gradients + momentum * past_velocity1
    l1_weights = l1_weights + velocity1
    past_velocity1 = velocity1
    
    velocity2 = learning_rate * l2_gradients + momentum * past_velocity2
    l2_weights = l2_weights + velocity2
    past_velocity2 = velocity2
    
    velocity_b1 = learning_rate * l1_biases_gradients + momentum * past_velocity_b1
    l1_biases = l1_biases + velocity_b1
    past_velocity_b1 = velocity_b1

    velocity_b2 = learning_rate * l1_biases_gradients + momentum * past_velocity_b2
    l2_biases = l2_biases + velocity_b2
    past_velocity_b2 = velocity_b2

    velocity3 = learning_rate * l3_gradients + momentum * past_velocity3
    l3_weights = l3_weights + velocity3
    past_velocity3 = velocity3


# # Here, we implement the Adam algorithm.

   
#     m_adam1 = beta_1 * m_adam1 + (1 - beta_1) * l1_gradients
#     v_adam1 = beta_2 * v_adam1 + (1 - beta_2) * np.power(l1_gradients, 2)
#     m_hat1 = m_adam1 / (1 - np.power(beta_1, ii))
#     v_hat1 = v_adam1 / (1 - np.power(beta_2, ii))
#     l1_weights = l1_weights + learning_rate * m_hat1 / (np.sqrt(v_hat1) + epsilon)
    
#     m_adam_b1 = beta_1 * m_adam_b1 + (1 - beta_1) * l1_biases_gradients
#     v_adam_b1 = beta_2 * v_adam_b1 + (1 - beta_2) * np.power(l1_biases_gradients, 2)
#     m_hat_b1 = m_adam_b1 / (1 - np.power(beta_1, ii))
#     v_hat_b1 = v_adam_b1 / (1 - np.power(beta_2, ii))
#     l1_biases = l1_biases + learning_rate * m_hat_b1 / (np.sqrt(v_hat_b1) + epsilon)
    
#     m_adam2 = beta_1 * m_adam2 + (1 - beta_1) * l2_gradients
#     v_adam2 = beta_2 * v_adam2 + (1 - beta_2) * np.power(l2_gradients, 2)
#     m_hat2 = m_adam2 / (1 - np.power(beta_1, ii))
#     v_hat2 = v_adam2 / (1 - np.power(beta_2, ii))
#     l2_weights = l2_weights + learning_rate * m_hat2 / (np.sqrt(v_hat2) + epsilon)
    
#     m_adam_b2 = beta_1 * m_adam_b2 + (1 - beta_1) * l2_biases_gradients
#     v_adam_b2 = beta_2 * v_adam_b2 + (1 - beta_2) * np.power(l2_biases_gradients, 2)
#     m_hat_b2 = m_adam_b2 / (1 - np.power(beta_1, ii))
#     v_hat_b2 = v_adam_b2 / (1 - np.power(beta_2, ii))
#     l2_biases = l2_biases + learning_rate * m_hat_b2 / (np.sqrt(v_hat_b2) + epsilon)
    
#     m_adam3 = beta_1 * m_adam3 + (1 - beta_1) * l3_gradients
#     v_adam3 = beta_2 * v_adam3 + (1 - beta_2) * np.power(l3_gradients, 2)
#     m_hat3 = m_adam3 / (1 - np.power(beta_1, ii))
#     v_hat3 = v_adam3 / (1 - np.power(beta_2, ii))
#     l3_weights = l3_weights + learning_rate * m_hat3 / (np.sqrt(v_hat3) + epsilon)
    
        
# Now, we calculate the outputs.
    l1_outs = relu(np.dot(l1_weights,cell_reps)+l1_biases)
    l2_outs = relu(np.dot(l2_weights,l1_outs)+l2_biases)
    l3_outs = np.dot(l3_weights, l2_outs)
    l3_outs = np.round(l3_outs, 16)
    
    for p in range(0, cells_num-1):
        for q in range (p+1, cells_num):
            if l3_outs[p] == l3_outs[q]:
                l3_outs[q] += np.random.rand(1,1)-0.5
   

    # We assigned the indexes based on the final layer outputs.
    sorted_outs = np.sort(l3_outs)
    indexes = []
    for i in range(0, cells_num):
        index = np.where(sorted_outs == l3_outs[i])
        indexes = np.append(indexes, index)

    #print(indexes)
    
    yijs = [] # Yijs based on the assigned indexes
    dijs = []
    for i in range(0, int(cells_num/2)):
        j = i + cells_num/2
        celli = np.where(indexes == i)
        cellj = np.where(indexes == j)

        yij = (xprbs[celli] + xprbs[cellj])/(prbs[celli] + prbs[cellj])
        yijs = np.append(yijs, yij)

        dij = x2prbs[celli] + yij**2 * prbs[celli] - yij *2 * xprbs[celli]\
            + x2prbs[cellj] + yij**2 * prbs[cellj] - yij *2 * xprbs[cellj]
        dijs = np.append(dijs, dij)

    distortion = np.round(sum(dijs), 16) # The Eve's distortion based on the assigned indexes
    print(distortion)

0.9480901976685344
0.9480901976685344
0.9480901976685344
0.9480901976685344
0.9480901976685344
0.9480901976685344
0.9480901976685344
0.9480901976685344
0.9480901976685344
0.9480901976685344
0.9480901976685344
0.9480901976685344
0.9480901976685344
0.9480901976685344
0.9480901976685344
0.9480901976685344
0.9480901976685344
0.9480901976685344
0.9480901976685344
0.9480901976685344
0.9480901976685344
0.9480901976685344
0.9480901976685344
0.9480901976685344
0.9480901976685344
0.9480901976685344
0.9480901976685344
0.9480901976685344
0.9480901976685344
0.9480901976685344
0.9480901976685344
0.9480901976685344
0.9480901976685344
0.9480901976685344
0.9480901976685344
0.9480901976685344
0.9480901976685344
0.9480901976685344
0.9480901976685344
0.9480901976685344
0.9480901976685344
0.9480901976685344
0.9480901976685344
0.9480901976685344
0.9480901976685344
0.9480901976685344
0.9480901976685344
0.9480901976685344
0.9480901976685344
0.9480901976685344
0.9480901976685344
0.9480901976685344
0.9480901976

In [1]:
# 79-character line limit
######### ######### ######### ######### ######### ######### ######### #########
%reset -f

import numpy as np # The NumPy library
#import math # The math module, np includes it
from scipy.integrate import quad # Method for integration in scipy.integrate sub-package


cells_num = 4

# The opitimized spaces for a Guassian distribution in a uniform quantizer(UQ).
# The sizes are for UQs for number of cells: 4, 8, 16, 32, 64, 128, 256.
all_size_gaus = np.array([0.9957, 0.5860, 0.3352, 0.1881, 0.1041, 0.0569, 0.0308])

cell_size = all_size_gaus[int(np.log2(cells_num))-2]

# np.arange has rounding error issue.
# boundaries_symm = np.arange(-(cells_num/2-1) * cell_size, # For symmetric distributions
#                              (cells_num/2) * cell_size,
#                             cell_size) 

boundaries_symm = np.linspace(-(cells_num/2-1) * cell_size, # For symmetric distributions
                               (cells_num/2-1) * cell_size,
                              cells_num-1)
n_inf = float("-inf")
p_inf = float("inf")

boundaries_symm = np.insert(boundaries_symm, 0, n_inf)
boundaries_symm = np.append(boundaries_symm, p_inf)

def pdf(x): # Defining the distribution
    gaus_std = 1
    gaus_mean = 0
    pdf = 1/(gaus_std*np.sqrt(2*np.pi)) * \
                  np.exp(-0.5*((x-gaus_mean)/gaus_std)**2) # Gaussian pdf
    return pdf

def xpdf(x):
    xpdf = x * pdf(x)
    return xpdf

def x2pdf (x):
    x2pdf = x * xpdf(x)
    return x2pdf

prbs = []
xprbs = []
x2prbs = []
cell_reps = []
for i in range (0, cells_num):
    cell_prb, integ_err = quad(pdf, boundaries_symm[i], boundaries_symm[i+1])
    prbs = np.append(prbs, cell_prb)
    
    cell_xprb, integ_err = quad(xpdf, boundaries_symm[i], boundaries_symm[i+1])
    xprbs = np.append(xprbs, cell_xprb)
    
    cell_x2prb, integ_err = quad(x2pdf, boundaries_symm[i], boundaries_symm[i+1])
    x2prbs = np.append(x2prbs, cell_x2prb)
    
    cell_rep = cell_xprb / cell_prb
    cell_reps = np.append(cell_reps, cell_rep)


# Constants: We prepare the constants for distortion calculations.

# Keras Implementation

In [288]:
# We use the Functional API option of Keras.

from keras import models
from keras import layers
from keras import Input
import keras.backend as K
import tensorflow as tf

l1_size = cells_num

input_tensor = Input(shape=(cells_num,))
l1_outs = layers.Dense(l1_size, activation = 'relu')(input_tensor)
print(l1_outs.shape)
l2_outs = layers.Dense(cells_num, activation = 'sigmoid')(l1_outs)

# def make_indexes(x):
#     raw_indexes = K.dot(x, cells_num)
#     indexes = K.round(raw_indexes)
#     for i in (0, cells_num-1):
#         for j in (i+1, cells_num):
#             if K.equal(indexes[i], indexes[j]):
                
#     return 

sorted_outs = layers.Lambda (sort_outs)(l2_outs)
#def distortion ()


(?, 8)


NameError: name 'asd' is not defined

# Data Augmentation

In [16]:
import random
from keras import models
from keras import layers
from keras import Input
from keras.utils import to_categorical


print(np.math.factorial(4))

distortions = []
indexes_set = []

for ii in range(0, 12):
    indexes = random.sample(range(8), 8)
    indexes = np.array(indexes)
    
    yijs = [] # Yijs based on the assigned indexes
    dijs = []
    for i in range(0, int(cells_num/2)):
        j = i + cells_num/2
        celli = np.where(indexes == i)
        cellj = np.where(indexes == j)

        yij = (xprbs[celli] + xprbs[cellj])/(prbs[celli] + prbs[cellj])
        yijs = np.append(yijs, yij)

        dij = x2prbs[celli] + yij**2 * prbs[celli] - yij *2 * xprbs[celli]\
            + x2prbs[cellj] + yij**2 * prbs[cellj] - yij *2 * xprbs[cellj]
        dijs = np.append(dijs, dij)

    distortion = np.round(sum(dijs), 16)
    distortions = np.append(distortions, distortion)
    indexes_set = np.append(indexes_set, indexes)
    indexes_set = np.reshape(
        indexes_set,(int(np.size(indexes_set)/cells_num), cells_num)) 
    
print(distortions)
print(indexes_set)



24
[0.62398884 0.56634292 0.77043103 0.63150229 0.39028457 0.9778991
 0.57253317 0.83175598 0.62398884 0.64544045 0.98116076 0.99303273]
[[1. 0. 6. 2. 5. 3. 7. 4.]
 [4. 1. 7. 0. 3. 5. 2. 6.]
 [5. 4. 7. 3. 2. 1. 6. 0.]
 [3. 4. 0. 1. 2. 5. 7. 6.]
 [1. 5. 0. 3. 7. 2. 6. 4.]
 [6. 1. 0. 3. 7. 2. 4. 5.]
 [0. 5. 3. 4. 1. 7. 2. 6.]
 [6. 7. 4. 2. 1. 0. 3. 5.]
 [0. 6. 2. 1. 7. 3. 4. 5.]
 [7. 6. 1. 2. 3. 0. 5. 4.]
 [3. 4. 2. 5. 7. 6. 1. 0.]
 [6. 0. 7. 1. 2. 4. 3. 5.]]


In [28]:
encoded = to_categorical(indexes_set[1])
print(encoded)
input = np.reshape(encoded,(cells_num * cells_num,))
np.shape(indexes_set)[0]

encoded_indexes_set = []
for i in range (0, np.shape(indexes_set)[0]):
    encoded_indexes = to_categorical(indexes_set[i])
    encoded_indexes = np.reshape(encoded_indexes,(cells_num * cells_num,))
    encoded_indexes_set = np.append(encoded_indexes_set, encoded_indexes)
    
encoded_indexes_set = np.reshape(
    int(np.size(encoded_indexes_set)/(cells_num*cells_num)), cells_num * cells_num)

        
    



[[0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]]


ValueError: cannot reshape array of size 1 into shape (64,)